In [0]:

path_store_daily1 = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.app-blackhorse.v1/dimension/"
df1 = spark.read.parquet(path_store_daily1)
df1.createOrReplaceTempView("past_26_weeks_black_horse_df")

path_store_daily2 = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.app-blackhorse-report.v1/fact/"
df2 = spark.read.parquet(path_store_daily2)
df2.createOrReplaceTempView("last_week_report_df")

path_store_daily3 = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.app-blackhorse.v1/dimension/"
df3 = spark.read.parquet(path_store_daily3)
df3.createOrReplaceTempView("fulfill_criteria_df")

path_store_daily4 = "s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app.v1/dimension/"
df4 = spark.read.parquet(path_store_daily4)
df4.createOrReplaceTempView("app_details_df")

path_store_daily5 = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.app-est.v1/fact/granularity=weekly/"
df5 = spark.read.parquet(path_store_daily5)
df5.createOrReplaceTempView("store_app_est_weekly_df")

path_store_daily6 = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.app-est-category-rank.v1/fact/granularity=weekly/"
df6 = spark.read.parquet(path_store_daily6)
df6.createOrReplaceTempView("store_app_rank_weekly_df")

path_store_daily7 = "s3://b2c-prod-data-pipeline-unified-dna/unified/app-tech.dna.in_app_dna_info_mapping.v1/dimension/"
df7 = spark.read.parquet(path_store_daily7)
df7.createOrReplaceTempView("app_dna_info_df")

path_store_daily8 = "s3://b2c-prod-data-pipeline-unified-dna/unified/app-tech.dna.dna_universal_company.v1/dimension/"
df8 = spark.read.parquet(path_store_daily8)
df8.createOrReplaceTempView("company_info_df")

path_store_daily9 = "s3://b2c-prod-data-pipeline-unified-dna/unified/app-tech.dna.dna_universal_company_office.v1/dimension/"
df9 = spark.read.parquet(path_store_daily9)
df9.createOrReplaceTempView("hq_company_country_df")

spark.sql("""
SELECT _black_horse_df.app_id, _black_horse_df.country_code, _black_horse_df.device_code, _black_horse_df.category_id, _black_horse_df.date, _black_horse_df.market_code, 
    _app_details_df.app_name, _app_details_df.release_date, _app_details_df.sub_category_id, 
    _store_app_est_weekly_df.free_app_download as current_download, _store_app_est_weekly_df.revenue as current_revenue, _store_app_est_weekly_df1.free_app_download as previous_week_download, _store_app_est_weekly_df1.revenue as previous_week_revenue,
    _store_app_rank_weekly_df.free_app_download as download_rank, _store_app_rank_weekly_df.revenue as revenue_rank,
    _app_dna_info_df.product_id, _app_dna_info_df.unified_product_id as unified_app_id, _app_dna_info_df.company_id, _app_dna_info_df.parent_company_id,
    _company_info_df.name as parent_company_name, _company_info_df1.name,
    _hq_company_country_df.country as hq_country_of_company
FROM
    (select black_horse_df.app_id, black_horse_df.country_code, black_horse_df.device_code, black_horse_df.category_id, '2019-11-23' date, IF(device_code in('ios-all'),'apple-store', 'google-play') market_code
        FROM
            (select app_id, country_code, device_code, category_id
                FROM
                    ((select fulfill_criteria_df.app_id, fulfill_criteria_df.country_code, fulfill_criteria_df.device_code, fulfill_criteria_df.category_id
                        FROM fulfill_criteria_df
                        WHERE date ='2019-11-23')
                    EXCEPT(
                        SELECT app_id, country_code, device_code, category_id from past_26_weeks_black_horse_df
                    ))
        UNION ALL
            SELECT app_id, country_code, device_code, category_id
            FROM last_week_report_df)
    AS black_horse_df)
AS  _black_horse_df


LEFT JOIN(
    SELECT id as app_id, name as app_name, release_date, market_code, category_id as sub_category_id from app_details_df)
    AS _app_details_df
    ON _black_horse_df.app_id = _app_details_df.app_id and _black_horse_df.market_code = _app_details_df.market_code

LEFT JOIN(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date
    FROM store_app_est_weekly_df
    WHERE date in ('2019-11-23') and country_code in ("WW", "US", "CN", "JP") and device_code in ("android-all", "ios-all"))
    AS _store_app_est_weekly_df
    ON _black_horse_df.app_id = _store_app_est_weekly_df.app_id and _black_horse_df.country_code = _store_app_est_weekly_df.country_code 
    and _black_horse_df.device_code = _store_app_est_weekly_df.device_code
    
LEFT JOIN(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date
    FROM store_app_est_weekly_df
    WHERE date in ('2019-11-16') and country_code in ("WW", "US", "CN", "JP") and device_code in ("android-all", "ios-all"))
    AS _store_app_est_weekly_df1
    ON _black_horse_df.app_id = _store_app_est_weekly_df1.app_id and _black_horse_df.country_code = _store_app_est_weekly_df1.country_code 
    and _black_horse_df.device_code = _store_app_est_weekly_df1.device_code

LEFT JOIN
(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date, category_id
    FROM store_app_rank_weekly_df
    WHERE date = '2019-11-23' and country_code in ("WW", "US", "CN", "JP") and device_code in ("android-all", "ios-all") 
        AND category_id in (400025, 400001, 100021, 100001)
) AS _store_app_rank_weekly_df
    ON _black_horse_df.date = _store_app_rank_weekly_df.date and _black_horse_df.app_id = _store_app_rank_weekly_df.app_id and _black_horse_df.category_id = _store_app_rank_weekly_df.category_id and _black_horse_df.country_code = _store_app_rank_weekly_df.country_code and _black_horse_df.device_code = _store_app_rank_weekly_df.device_code 

LEFT JOIN(
    SELECT product_id, unified_product_id, company_id, parent_company_id
    FROM app_dna_info_df)
    AS _app_dna_info_df
    ON _black_horse_df.app_id = _app_dna_info_df.product_id
    
LEFT JOIN(
    SELECT id, name
    FROM company_info_df)
    AS _company_info_df
    ON _app_dna_info_df.parent_company_id = _company_info_df.id
    
LEFT JOIN(
    SELECT id, name
    FROM company_info_df)
    AS _company_info_df1
    ON _app_dna_info_df.company_id = _company_info_df1.id

LEFT JOIN(
    SELECT universal_company_id, country
    FROM hq_company_country_df)
    AS _hq_company_country_df
    ON _app_dna_info_df.company_id = _hq_company_country_df.universal_company_id
""").createOrReplaceTempView("black_horse_dna_df")


spark.sql("""
SELECT app_id, country_code, device_code, category_id, count(distinct date) as count_in_chart, min(date) as first_week_in_chart, max(date) as last_week_in_chart
FROM
    (SELECT black_horse_dna_df.app_id, black_horse_dna_df.country_code, black_horse_dna_df.device_code, black_horse_dna_df.category_id, _past_fulfill_criteria_df.date
    FROM black_horse_dna_df
    LEFT JOIN
    (
        SELECT app_id, country_code, device_code, category_id, date
        FROM fulfill_criteria_df
        WHERE date >= "2019-01-05" and date <= "2019-11-23"
    ) AS _past_fulfill_criteria_df
    ON black_horse_dna_df.app_id = _past_fulfill_criteria_df.app_id and black_horse_dna_df.country_code = _past_fulfill_criteria_df.country_code and black_horse_dna_df.device_code = _past_fulfill_criteria_df.device_code and black_horse_dna_df.category_id = _past_fulfill_criteria_df.category_id )
AS _df
GROUP BY app_id, country_code, device_code, category_id 

""").createOrReplaceTempView("df")

spark.sql("""
SELECT black_horse_dna_df.app_id, black_horse_dna_df.country_code, black_horse_dna_df.device_code, black_horse_dna_df.category_id, black_horse_dna_df.date, black_horse_dna_df.market_code, black_horse_dna_df.app_name, black_horse_dna_df.release_date, black_horse_dna_df.sub_category_id, black_horse_dna_df.current_download, black_horse_dna_df.current_revenue, black_horse_dna_df.previous_week_download, black_horse_dna_df.previous_week_revenue, black_horse_dna_df.download_rank, black_horse_dna_df.revenue_rank, black_horse_dna_df.product_id, black_horse_dna_df.unified_app_id, black_horse_dna_df.company_id, black_horse_dna_df.parent_company_id, black_horse_dna_df.parent_company_name, black_horse_dna_df.name as company_name, black_horse_dna_df.hq_country_of_company,
    IF(black_horse_dna_df.category_id in ('100021', '400025'),'free', 'revenue') feed,
    _df.count_in_chart, _df.last_week_in_chart, _df.first_week_in_chart
FROM
    (SELECT _black_horse_df.app_id, _black_horse_df.country_code, _black_horse_df.device_code, _black_horse_df.category_id, _black_horse_df.date, _black_horse_df.market_code, 
        _app_details_df.app_name, _app_details_df.release_date, _app_details_df.sub_category_id, 
        _store_app_est_weekly_df.free_app_download as current_download, _store_app_est_weekly_df.revenue as current_revenue, _store_app_est_weekly_df1.free_app_download as previous_week_download, _store_app_est_weekly_df1.revenue as previous_week_revenue,
        _store_app_rank_weekly_df.free_app_download as download_rank, _store_app_rank_weekly_df.revenue as revenue_rank,
        _app_dna_info_df.product_id, _app_dna_info_df.unified_product_id as unified_app_id, _app_dna_info_df.company_id, _app_dna_info_df.parent_company_id,
        _company_info_df.name as parent_company_name, _company_info_df1.name,
        _hq_company_country_df.country as hq_country_of_company
    FROM
        (select black_horse_df.app_id, black_horse_df.country_code, black_horse_df.device_code, black_horse_df.category_id, '2019-11-23' date, IF(device_code in('ios-all'),'apple-store', 'google-play') market_code
            FROM
                (select app_id, country_code, device_code, category_id
                    FROM
                        ((select fulfill_criteria_df.app_id, fulfill_criteria_df.country_code, fulfill_criteria_df.device_code, fulfill_criteria_df.category_id
                            FROM fulfill_criteria_df
                            WHERE date ='2019-11-23')
                        EXCEPT(
                            SELECT app_id, country_code, device_code, category_id from past_26_weeks_black_horse_df
                        ))
        UNION ALL
            SELECT app_id, country_code, device_code, category_id
            FROM last_week_report_df)
    AS black_horse_df)
AS  _black_horse_df


LEFT JOIN(
    SELECT id as app_id, name as app_name, release_date, market_code, category_id as sub_category_id from app_details_df)
    AS _app_details_df
    ON _black_horse_df.app_id = _app_details_df.app_id and _black_horse_df.market_code = _app_details_df.market_code

LEFT JOIN(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date
    FROM store_app_est_weekly_df
    WHERE date in ('2019-11-23') and country_code in ("WW", "US", "CN", "JP") and device_code in ("android-all", "ios-all"))
    AS _store_app_est_weekly_df
    ON _black_horse_df.app_id = _store_app_est_weekly_df.app_id and _black_horse_df.country_code = _store_app_est_weekly_df.country_code 
    and _black_horse_df.device_code = _store_app_est_weekly_df.device_code
    
LEFT JOIN(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date
    FROM store_app_est_weekly_df
    WHERE date in ('2019-11-16') and country_code in ("WW", "US", "CN", "JP") and device_code in ("android-all", "ios-all"))
    AS _store_app_est_weekly_df1
    ON _black_horse_df.app_id = _store_app_est_weekly_df1.app_id and _black_horse_df.country_code = _store_app_est_weekly_df1.country_code 
    and _black_horse_df.device_code = _store_app_est_weekly_df1.device_code

LEFT JOIN(
    SELECT app_id, country_code, device_code, free_app_download, revenue, date, category_id
    FROM store_app_rank_weekly_df
    WHERE date = '2019-11-23' AND country_code in ("WW", "US", "CN", "JP") AND device_code in ("android-all", "ios-all") 
        AND category_id in (400025, 400001, 100021, 100001))
    AS _store_app_rank_weekly_df
    ON _black_horse_df.date = _store_app_rank_weekly_df.date AND _black_horse_df.app_id = _store_app_rank_weekly_df.app_id AND _black_horse_df.category_id = _store_app_rank_weekly_df.category_id AND _black_horse_df.country_code = _store_app_rank_weekly_df.country_code AND _black_horse_df.device_code = _store_app_rank_weekly_df.device_code 

LEFT JOIN(
    SELECT product_id, unified_product_id, company_id, parent_company_id
    FROM app_dna_info_df)
    AS _app_dna_info_df
    ON _black_horse_df.app_id = _app_dna_info_df.product_id
    
LEFT JOIN(
    SELECT id, name
    FROM company_info_df)
    AS _company_info_df
    ON _app_dna_info_df.parent_company_id = _company_info_df.id
    
LEFT JOIN(
    SELECT id, name
    FROM company_info_df)
    AS _company_info_df1
    ON _app_dna_info_df.company_id = _company_info_df1.id

LEFT JOIN(
    SELECT universal_company_id, country, type
    FROM hq_company_country_df)
    AS _hq_company_country_df
    ON _app_dna_info_df.company_id = _hq_company_country_df.universal_company_id and _hq_company_country_df.type = "HQ")
AS black_horse_dna_df

INNER JOIN(
    SELECT app_id, country_code, device_code, category_id, count_in_chart, last_week_in_chart, first_week_in_chart FROM df
)
AS _df
ON black_horse_dna_df.app_id = _df.app_id and black_horse_dna_df.country_code = _df.country_code and black_horse_dna_df.device_code = _df.device_code and black_horse_dna_df.category_id = _df.category_id

""").createOrReplaceTempView("_black_horse_report_df")

spark.sql("""
         select distinct app_id, country_code, device_code, category_id, first_week_in_chart, last_week_in_chart, count_in_chart, concat(cast((count_in_chart/(((datediff(last_week_in_chart,first_week_in_chart))/7)+1))*100 as int), "%") as ratio
         FROM _black_horse_report_df
""").createOrReplaceTempView("df2")

spark.sql("""
    SELECT 
           black_horse_report_df.feed, df3.app_id, black_horse_report_df.app_name, black_horse_report_df.unified_app_id, black_horse_report_df.release_date,
           black_horse_report_df.country_code,  df3.device_code, df3.category_id, black_horse_report_df.sub_category_id, black_horse_report_df.current_download, 
           black_horse_report_df.current_revenue, black_horse_report_df.previous_week_download, black_horse_report_df.previous_week_revenue, 
           black_horse_report_df.download_rank, black_horse_report_df.revenue_rank, black_horse_report_df.company_id, black_horse_report_df.company_name,
           black_horse_report_df.parent_company_id, black_horse_report_df.parent_company_name, black_horse_report_df.hq_country_of_company, 
           df3.first_week_in_chart, df3.last_week_in_chart, df3.count_in_chart, df3.ratio 
    FROM (SELECT * FROM df2) AS df3
    LEFT JOIN (SELECT _black_horse_report_df.feed, _black_horse_report_df.app_name, _black_horse_report_df.unified_app_id, _black_horse_report_df.release_date,
           _black_horse_report_df.country_code, _black_horse_report_df.sub_category_id, _black_horse_report_df.current_download, 
           _black_horse_report_df.current_revenue, _black_horse_report_df.previous_week_download, _black_horse_report_df.previous_week_revenue, 
           _black_horse_report_df.download_rank, _black_horse_report_df.revenue_rank, _black_horse_report_df.company_id, _black_horse_report_df.company_name,
           _black_horse_report_df.parent_company_id, _black_horse_report_df.parent_company_name, _black_horse_report_df.hq_country_of_company,
           _black_horse_report_df.app_id,  _black_horse_report_df.device_code, _black_horse_report_df.country_code, _black_horse_report_df.category_id, 
           _black_horse_report_df.last_week_in_chart
           FROM _black_horse_report_df) AS black_horse_report_df
    ON df3.app_id = black_horse_report_df.app_id and df3.device_code = black_horse_report_df.device_code and df3.country_code = black_horse_report_df.country_code and 
       df3.category_id = black_horse_report_df.category_id and df3.last_week_in_chart = black_horse_report_df.last_week_in_chart
    ORDER BY df3.app_id, black_horse_report_df.country_code, df3.device_code, df3.category_id
""").show(2000)

In [0]:

spark.sql("""
    SELECT 
           black_horse_report_df.feed, df3.app_id, black_horse_report_df.app_name, black_horse_report_df.unified_app_id, black_horse_report_df.release_date,
           black_horse_report_df.country_code,  df3.device_code, df3.category_id, black_horse_report_df.sub_category_id, black_horse_report_df.current_download, 
           black_horse_report_df.current_revenue, black_horse_report_df.previous_week_download, black_horse_report_df.previous_week_revenue, 
           black_horse_report_df.download_rank, black_horse_report_df.revenue_rank, black_horse_report_df.company_id, black_horse_report_df.company_name,
           black_horse_report_df.parent_company_id, black_horse_report_df.parent_company_name, black_horse_report_df.hq_country_of_company, 
           df3.first_week_in_chart, df3.last_week_in_chart, df3.count_in_chart, df3.ratio 
    FROM (SELECT * FROM df2) AS df3
    LEFT JOIN (SELECT black_horse_report_df.feed, black_horse_report_df.app_name, black_horse_report_df.unified_app_id, black_horse_report_df.release_date,
           black_horse_report_df.country_code, black_horse_report_df.sub_category_id, black_horse_report_df.current_download, 
           black_horse_report_df.current_revenue, black_horse_report_df.previous_week_download, black_horse_report_df.previous_week_revenue, 
           black_horse_report_df.download_rank, black_horse_report_df.revenue_rank, black_horse_report_df.company_id, black_horse_report_df.company_name,
           black_horse_report_df.parent_company_id, black_horse_report_df.parent_company_name, black_horse_report_df.hq_country_of_company,
           black_horse_report_df.app_id,  black_horse_report_df.device_code, black_horse_report_df.country_code, black_horse_report_df.category_id, 
           black_horse_report_df.last_week_in_chart
           FROM black_horse_report_df) AS black_horse_report_df
    ON df3.app_id = black_horse_report_df.app_id and df3.device_code = black_horse_report_df.device_code and df3.country_code = black_horse_report_df.country_code and 
       df3.category_id = black_horse_report_df.category_id and df3.last_week_in_chart = black_horse_report_df.last_week_in_chart
    ORDER BY df3.app_id, black_horse_report_df.country_code, df3.device_code, df3.category_id
""").show(2000)